# Criação de Ensembles

## 0.0 Configurações

In [2]:
# configurações
import importlib #quando necessário reimportar pacotes

import sys
sys.path.append("../ml-project-template")

from config import *
import utils
import data_manager as data_mgr
import model_tuner as mod_tuner
import ensemble_creator as ens_crt

In [3]:
data_manager = data_mgr.DataManager()

In [4]:
_ , raw_test_df = data_manager.load_raw_data()

Treino - formato: (593994, 13)
Teste  - formato: (254569, 12)


In [6]:
train_v0, test_v0 = data_manager.load_processed_data(feature_set = "v0_sem_encoder")
train_v1, test_v1 = data_manager.load_processed_data(feature_set = "v1_basico")
train_v2, test_v2 = data_manager.load_processed_data(feature_set = "v2_normalizado")
train_v3, test_v3 = data_manager.load_processed_data(feature_set = "v3_poly")
train_v4, test_v4 = data_manager.load_processed_data(feature_set = "v4_norm_poly")

 Carregando features: v0_sem_encoder
 FeatureSet info: 12 features
 Carregando features: v1_basico
 FeatureSet info: 61 features
 Carregando features: v2_normalizado
 FeatureSet info: 61 features
 Carregando features: v3_poly
 FeatureSet info: 76 features
 Carregando features: v4_norm_poly
 FeatureSet info: 27 features


In [7]:
X_v0, y_v0 = data_manager.split_features_target(train_v0)
X_v1, y_v1 = data_manager.split_features_target(train_v1)
X_v2, y_v2 = data_manager.split_features_target(train_v2)
X_v3, y_v3 = data_manager.split_features_target(train_v3)
X_v4, y_v4 = data_manager.split_features_target(train_v4)

 Features: (593994, 11), Target: (593994,)
 Features: (593994, 60), Target: (593994,)
 Features: (593994, 60), Target: (593994,)
 Features: (593994, 75), Target: (593994,)
 Features: (593994, 26), Target: (593994,)


In [8]:
cat_opt_v0 = utils.load_model("cat_opt_v0")
lgb_opt_v1 = utils.load_model("lgb_opt_v1")
xgb_opt_v1 = utils.load_model("xgb_opt_v1")
ngb_opt_v2 = utils.load_model("ngb_opt_v2")
lgb_opt_v3 = utils.load_model("lgb_opt_v3")
cat_opt_v4 = utils.load_model("cat_opt_v4")

## 1.0 Ensemble A (cat_opt_v0, lgb_opt_v1, xgb_opt_v1)

In [26]:
models = [ cat_opt_v0, lgb_opt_v1, xgb_opt_v1]
X_train_list = [X_v0, X_v1, X_v1]

In [27]:
ensembleA = ens_crt.EnsembleCreator(models, X_train_list, y_v0)

In [28]:
ensembleA.optimize_weights_cv('roc_auc', min_weight = 0.05)

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging

array([0.3155801 , 0.35108612, 0.33333378])

In [29]:
X_test_list = [test_v0, test_v1, test_v1]

In [30]:
y_pred_proba = ensembleA.predict_proba(X_test_list)

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


In [31]:
submission_ensembleA = pd.DataFrame({"id": raw_test_df["id"], "loan_paid_back": y_pred_proba})
utils.save_submission(submission_ensembleA, "ensembleA")

## 2.0 Ensemble B (cat_opt_v0, lgb_opt_v1, xgb_opt_v1. ngb_opt_v2)

In [9]:
models = [cat_opt_v0, lgb_opt_v1, xgb_opt_v1, ngb_opt_v2]
X_train_list = [X_v0, X_v1, X_v1, X_v2]

In [12]:
ensembleB = ens_crt.EnsembleCreator(models, X_train_list, y_v0.astype(int))

In [13]:
ensembleB.optimize_weights_cv('roc_auc', min_weight = 0.05)

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


C:\Users\lucia\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_slsqp_py.py:434: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\lucia\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_slsqp_py.py:438: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)
C:\Users\lucia\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_slsqp_py.py:492: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  a_eq = vstack([con['jac'](x, *con['args'])


[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


C:\Users\lucia\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_slsqp_py.py:434: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
CV Scores: 0.9311 ± 0.0019


array([0.16111628, 0.45406227, 0.20360637, 0.18121509])

In [14]:
X_test_list = [test_v0, test_v1, test_v1, test_v2]
y_pred_proba = ensembleB.predict_proba(X_test_list)

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


In [15]:
submission_ensembleB = pd.DataFrame({"id": raw_test_df["id"], "loan_paid_back": y_pred_proba})
utils.save_submission(submission_ensembleB, "ensembleB")

## 3.0 Ensemble C (cat_opt_v0, lgb_opt_v1)

In [16]:
models_C = [cat_opt_v0, lgb_opt_v1]
X_train_list_C = [X_v0, X_v1]

In [21]:
ensembleC = ens_crt.EnsembleCreator(models_C, X_train_list_C, y_v0)
ensembleC.optimize_weights_cv('roc_auc', min_weight = 0.05)

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging

array([0.5, 0.5])

In [19]:
X_test_list_C = [test_v0, test_v1]
y_pred_proba_C = ensembleC.predict_proba(X_test_list_C)

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


In [20]:
submission_ensembleC = pd.DataFrame({"id": raw_test_df["id"], "loan_paid_back": y_pred_proba_C})
utils.save_submission(submission_ensembleC, "ensembleC")

## 3.0 Ensemble D (cat_opt_v0, lgb_opt_v1, xgb_opt_v1, lgb_opt_v3)

In [8]:
models_D = [cat_opt_v0, lgb_opt_v1, xgb_opt_v1, lgb_opt_v3]
X_train_list_D = [X_v0, X_v1, X_v1, X_v3]

In [9]:
ensembleD = ens_crt.EnsembleCreator(models_D, X_train_list_D, y_v0)

In [10]:
ensembleD.optimize_weights_cv('roc_auc', min_weight = 0.05)

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging

array([0.23655615, 0.2705445 , 0.22864831, 0.26425104])

In [11]:
X_test_list_D = [test_v0, test_v1, test_v1, test_v3]
y_pred_proba_D = ensembleD.predict_proba(X_test_list_D)

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


In [12]:
submission_ensembleD = pd.DataFrame({"id": raw_test_df["id"], "loan_paid_back": y_pred_proba_D})
utils.save_submission(submission_ensembleD, "ensembleD")

## 4.0 Ensemble E (cat_opt_v4, lgb_opt_v3)

In [10]:
models_E = [cat_opt_v4, lgb_opt_v3]
X_train_list_E = [X_v4, X_v3]

In [11]:
ensembleE = ens_crt.EnsembleCreator(models_E, X_train_list_E, y_v0)
ensembleE.optimize_weights_cv('roc_auc', min_weight = 0.05)

[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging

array([0.49408233, 0.50591767])

In [12]:
X_test_list_E = [test_v4, test_v3]
y_pred_proba_E = ensembleE.predict_proba(X_test_list_E)
submission_ensembleE = pd.DataFrame({"id": raw_test_df["id"], "loan_paid_back": y_pred_proba_E})
utils.save_submission(submission_ensembleE, "ensembleE")

[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


## 5.0 Ensemble F (cat_opt_v0, lgb_opt_v1, xgb_opt_v1,cat_opt_v4, lgb_opt_v3)

In [14]:
models_F = [cat_opt_v0, lgb_opt_v1, xgb_opt_v1, cat_opt_v4, lgb_opt_v3]
X_train_list_F = [X_v0, X_v1, X_v1, X_v4, X_v3]

In [16]:
ensembleF = ens_crt.EnsembleCreator(models_F, X_train_list_F, y_v0)
ensembleF.optimize_weights_cv('roc_auc', min_weight = 0.00)

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging

array([0.20236707, 0.20236707, 0.1964494 , 0.1964494 , 0.20236707])

In [17]:
X_test_list_F = [test_v0, test_v1, test_v1, test_v4, test_v3]
y_pred_proba_F = ensembleF.predict_proba(X_test_list_F)
submission_ensembleF = pd.DataFrame({"id": raw_test_df["id"], "loan_paid_back": y_pred_proba_F})
utils.save_submission(submission_ensembleF, "ensembleF")

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


## 6.0 Ensemble G (lgb_opt_v1, xgb_opt_v1, cat_opt_v4, lgb_opt_v3)

In [18]:
models_G = [lgb_opt_v1, xgb_opt_v1, cat_opt_v4, lgb_opt_v3]
X_train_list_G = [X_v1, X_v1, X_v4, X_v3]

In [19]:
ensembleG = ens_crt.EnsembleCreator(models_G, X_train_list_G, y_v0)
ensembleG.optimize_weights_cv('roc_auc', min_weight = 0.00)

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging

array([0.35334487, 0.17633336, 0.2455824 , 0.22473938])

In [20]:
X_test_list_G = [test_v1, test_v1, test_v4, test_v3]
y_pred_proba_G = ensembleG.predict_proba(X_test_list_G)
submission_ensembleG = pd.DataFrame({"id": raw_test_df["id"], "loan_paid_back": y_pred_proba_G})
utils.save_submission(submission_ensembleG, "ensembleG")

[LightGBM] [Warning] feature_fraction is set=0.7188853485951814, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7188853485951814
[LightGBM] [Warning] bagging_fraction is set=0.9803426366722742, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803426366722742
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


## Ensemble H (xgb_opt_v1, cat_opt_v4, lgb_opt_v3)

In [22]:
models_H = [xgb_opt_v1, cat_opt_v4, lgb_opt_v3]
X_train_list_H = [X_v1, X_v4, X_v3]

In [23]:
ensembleH = ens_crt.EnsembleCreator(models_H, X_train_list_H, y_v0)
ensembleH.optimize_weights_cv('roc_auc', min_weight = 0.00)

[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging

array([0.33334198, 0.33331896, 0.33333907])

In [25]:
X_test_list_H = [test_v1, test_v4, test_v3]
y_pred_proba_H = ensembleH.predict_proba(X_test_list_H)
submission_ensembleH = pd.DataFrame({"id": raw_test_df["id"], "loan_paid_back": y_pred_proba_H})
utils.save_submission(submission_ensembleH, "ensembleH")

[LightGBM] [Warning] feature_fraction is set=0.7176731445490705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7176731445490705
[LightGBM] [Warning] bagging_fraction is set=0.8395415387770279, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8395415387770279
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
